In [1]:
import numpy as np
import random

import tensorflow as tf
from tensorflow.keras.datasets import mnist

In [2]:
class GenNN:
    def __init__(self,sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.uniform(size=(y,1)) for y in sizes[1:]] #for a network with n layer, we have n-1 bias vector
        self.weights = [np.random.uniform(size=(y,x)) for x,y in zip(sizes[:-1],sizes[1:])]
        self.chromosom = np.concatenate([w.flatten() for w in self.weights]+[b.flatten() for b in self.biases])
        self.N_gens = len(self.chromosom)
        
    @staticmethod
    def sigmoid(z):
        return 1.0/(1.0+np.exp(-z))
    
    def unflatten_weights(self):
        weights = []
        biases = []
        idx = 0
        sizes = self.sizes
        for i in range(len(sizes) - 1):
            weight_size = sizes[i]*sizes[i + 1]
            weights.append(self.chromosom[idx:idx + weight_size].reshape((sizes[i+1], sizes[i])))
            idx += weight_size
            
            # Extracting biases for each layer
            biases_shape = (sizes[i + 1], 1)
            biases.append(self.chromosom[idx:idx + sizes[i + 1]].reshape(biases_shape))
            idx += sizes[i + 1]
            
        self.weights=weights
        self.biases = biases

            
    def feedforward(self,a):
        """
        Return the output of the network with input "a"
        """
        for b,w in zip(self.biases,self.weights):
            a=GenNN.sigmoid(np.dot(w,a)+b.flatten())
            
        return a      

# MNIST Data

In [3]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Optionally, you can normalize the pixel values to be in the range [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

training_data = [(x_train[i].flatten(),y_train[i]) for i in range(60000)]
test_data = [(x_test[i].flatten(),y_test[i]) for i in range(10000)]

In [4]:
training_data[1][0][1]

0.0

# 1. Generate Population

In [5]:
architecture = [784,20,10]

def fitness(ind,data=training_data):
    return sum([(np.argmax(ind.feedforward(x))-y)**2 for x,y in training_data])/len(data)

In [42]:
initial_pop = [GenNN(architecture) for i in range(20)]

In [14]:
pop_fitness = []
for i,individual in enumerate(initial_pop):
    print(i,end='')
    pop_fitness.append(fitness(individual))

012345678910111213141516171819202122232425262728293031323334353637383940414243444546474849

# 2. Selection

In [46]:
#deterministic selection
def select(individuals,pn = 0.3):
    """
    pn : % or integer
    """
    n = int(len(individuals)*pn) if pn<1 else pn
        
    pop_fitness = [fitness(individual) for individual in individuals]
    top_n = sorted(range(len(pop_fitness)), key=lambda i: pop_fitness[i], reverse=True)[:n]
    worst_n = sorted(range(len(pop_fitness)), key=lambda i: pop_fitness[i], reverse=True)[-n:]
    return top_n,worst_n

In [40]:
top_n,worst_n = select(initial_pop)

KeyboardInterrupt: 

# 3. Cross Over

Here we will be implementing Mean-based-Crossover, a technique presented in the following paper : [GGA-MLP: A Greedy Genetic Algorithm to Optimize Weights and Biases in Multilayer Perceptron](https://en.wikipedia.org/wiki/Crossover_(genetic_algorithm)#:~:text=In%20genetic%20algorithms%20and%20evolutionary,parents%20to%20generate%20new%20offspring.)

In [44]:
S = set()

def MBC(pop,top_n_index,worst_n_index,N_off_springs):
    """
    input :
        top_n_index : Index of top N chromosm of the list of population
        worst_n_index : Index of worst N chromosm of the list of population
        N_off_springs : Number of off springs desired to be created
    ouput :
        off_springs : Generated chromosoms from Cross Over
    """
    
    #Calculate the gene-wisse mean of top N chromosoms
    stacked_arrays = np.stack([pop[i].chromosom for i in top_n_index],axis=0)
    C_mean = np.mean(stacked_arrays,axis=0)
    
    off_springs = []
    for _ in range(N_off_springs//2) :
        #Select a random C_bf chromosom that's not chosen yet
        C_bf_index = random.choice(top_n_index)
        while C_bf_index in S:
            C_bf_index = random.choice(top_n_index)
        C_bf = pop[C_bf_index].chromosom

        #Calculate the absolute difference between genes of C_bf and C_mean
        C_diff = np.abs(C_bf-C_mean)

        #
        scores = [] #Scores of each C_diff
        for j in worst_n_index:
            C_diff_j = np.abs(pop[j].chromosom-C_mean) #calculate the difference chromosom
            scores.append(np.sum(C_diff_j<C_diff)) #append the score value to scores (score is an indicator of how significantly Cj can improve the fitness of the chromosome Cbf .)

        #Select the best chromosom for crossover
        selected_index = np.argmax(scores) 
        C_hs = pop[selected_index].chromosom
        C_hs_diff = np.abs(C_hs-C_mean) #Calculate its difference

        #Crossover is now performed between C_bf and C_hs by interchanging the genes for which C [k]_hs_diff <C[k]_diff . 
        c_indices = np.where(C_hs_diff < C_diff)[0]
        C_bf[c_indices], C_hs[c_indices] = C_hs[c_indices], C_bf[c_indices]

        #Add C_bf to off springs and its index to S
        S.add(selected_index)
        off_springs.append(C_bf)
        off_springs.append(C_hs)
        
    #Turn off_springs into object of our class GenNN
    off_springs_objects = [GenNN(architecture) for i in range(N_off_springs)]
    for i,c in enumerate(off_springs):
        off_springs_objects[i].chromosom=c
        off_springs_objects[i].unflatten_weights()

    return off_springs_objects

In [51]:
def GreedyMutation(pop,top_n_index,worst_n_index,n,Pm = 0.3):
    """
     input :
            top_n_index : Index of top N chromosm of the list of population
            worst_n_index : Index of worst N chromosm of the list of population
            Pm : Mutation probability
            n : Number of greedy mutations
    """
    #Calculate the gene-wisse mean of top N chromosoms
    stacked_arrays = np.stack([pop[i].chromosom for i in top_n_index],axis=0)
    C_mean = np.mean(stacked_arrays,axis=0)
    
    for _ in range(n):
        #Select a random C_j from the worst chromosom for mutation
        C_j_index = random.choice(worst_n_index)
        C_j = pop[C_j_index].chromosom

        #iterate over the selected chromosom genes
        for i,gene in enumerate(C_j):
            R = random.random() #Generate a random number
            if R>Pm:
                d = abs(gene-C_mean[i])*random.random()
                C_j[i] -= d
        pop[C_j_index].chromosom = C_j      
    
    

# ALL TOGETHER

In [59]:
pop = initial_pop
N_pop = len(pop)
N_MBC_GM = int(N_pop*0.5)
N_elitism = int(N_pop*0.3)
N_rand = int(N_pop*0.2)
"""
At each iteration we will have :
30% using elitism
50% MBC and GreedyMutation
20% random
"""
for i in range(20):
    print("___GENERATION_N_",i)
    
    #Choose 30% by elitism
    new_pop = random.sample(pop, k=N_elitism)
    not_sampled = [c for c in pop if c not in new_pop] #We need to retain unsampled indivduals
    
    #Generate 20% random
    random_indiv = [GenNN(architecture) for _ in range(N_rand)]
    new_pop += random_indiv
    
    #Generate off springs using MBC on the 70% remaining (unsampled)
    print("Calculing top_n and worst_n ...")
    top_n,worst_n = select(not_sampled)
    print(top_n,worst_n,"***",len(not_sampled))
    off_springs = MBC(not_sampled,top_n,worst_n,N_MBC_GM)
    
    #Apply greedy mutation to off_springs
    GreedyMutation(off_springs,top_n,worst_n,N_MBC_GM,Pm = 0.3)
    new_pop += off_springs
            
    
    #update population (the new generation)
    print(len(new_pop))
    pop = new_pop
    print('________________________________________________________')
    

#Better fintness so far:
best = float("+inf")
for C in off_springs:
    C_fitness = fitness(C)
    if C_fitness<best:
        best = C_fitness
print("Best Fintess : ",best)

___GENERATION_N_ 0
Calculing top_n and worst_n ...
[12, 4, 3, 6] [10, 0, 11, 2] *** 14
30
________________________________________________________
___GENERATION_N_ 1
Calculing top_n and worst_n ...
[2, 5, 8, 10, 12, 14, 16] [1, 0, 6, 11, 13, 19, 22] *** 24


IndexError: list index out of range